## Basic local/development deploy using runserver (DEBUG=True)
Deploy onto an Ubuntu 18.04 server

https://www.digitalocean.com/community/tutorials/systemd-essentials-working-with-services-units-and-the-journal

https://www.digitalocean.com/community/tutorials/how-to-set-up-ssh-keys-on-ubuntu-1604

https://www.digitalocean.com/community/tutorials/how-to-install-mysql-on-ubuntu-18-04

https://github.com/joke2k/django-environ/blob/develop/README.rst

https://www.digitalocean.com/community/tutorials/how-to-set-up-django-with-postgres-nginx-and-gunicorn-on-ubuntu-18-04

https://www.digitalocean.com/community/tutorials/how-to-set-up-object-storage-with-django

https://www.digitalocean.com/community/tutorials/how-to-install-nginx-on-ubuntu-18-04

https://www.digitalocean.com/community/tutorials/how-to-secure-nginx-with-let-s-encrypt-on-ubuntu-18-04

https://certbot.eff.org/docs/install.html#docker-user

https://certbot-dns-digitalocean.readthedocs.io/en/latest/

https://www.digitalocean.com/community/tutorials/how-to-create-a-self-signed-ssl-certificate-for-nginx-in-ubuntu-18-04

https://realpython.com/caching-in-django-with-redis/

https://realpython.com/caching-in-django-with-redis/

https://niwinz.github.io/django-redis/latest/

https://micropyramid.com/blog/how-to-monitor-django-application-live-events-with-sentry/

https://docs.sentry.io/clients/python/integrations/django/

#### misc

https://www.digitalocean.com/community/tutorials/how-to-use-sshfs-to-mount-remote-file-systems-over-ssh

https://www.digitalocean.com/community/tutorials/how-to-configure-custom-connection-options-for-your-ssh-client

In [ ]:
# ENV copy to a file, edit, source
export APP_HOST=edcdev.clinicedc.org
export APP_NAME=ambition
export APP_USER=ambition
export DJANGO_EXPORT_FOLDER=/home/$APP_USER/export
export DJANGO_ETC_FOLDER=/home/$APP_USER/.etc/$APP_NAME
export DJANGO_KEY_FOLDER=$DJANGO_ETC_FOLDER/crypto_fields
export DJANGO_LOG_FOLDER=/home/$APP_USER/log
export DJANGO_STATIC_FOLDER=/home/$APP_USER/static
export MYSQL_DATABASE=ambition
export MYSQL_USER=edc
export MYSQL_USER_PASSWORD=mai9Chai,  # need a password
export REPO=https://github.com/ambition-trial/ambition.git
export VENV=ambition


In [ ]:
# prepare the droplet
# as root, create a non-root account and set up keys
adduser $APP_USER
# add to sudo
usermod -aG sudo $APP_USER
# copy keys to the new account
mkdir /home/$APP_USER/.ssh
cp .ssh/authorized_keys /home/$APP_USER/.ssh
chown $APP_USER:$APP_USER -R /home/$APP_USER/.ssh
chmod 700 /home/$APP_USER/.ssh
chmod 600 /home/$APP_USER/.ssh/authorized_keys

In [ ]:
# prepare the OS (Bionic)
# login as non-root account ambition
# install dependencies
sudo apt-get update
sudo apt-get -y upgrade
sudo apt-get -y install mysql-server # if needed
sudo apt-get -y install mysql-client-5.7 libmysqlclient-dev libcups2-dev ipython3 python3-pip python3-dev python3-venv python3-cups python3-venv redis-server

In [ ]:
# prepare mysql-server
# Note: on Bionic access root with `sudo mysql`

# login to MySQL as non-root account

# secure MySQL installation
sudo mysql_secure_installation

# load timezones into MySQL
mysql_tzinfo_to_sql /usr/share/zoneinfo | sudo mysql mysql

# create a MySQL database for the app
echo "CREATE DATABASE $MYSQL_DATABASE CHARACTER SET utf8;" | sudo mysql
echo "CREATE DATABASE $MYSQL_DATABASE CHARACTER SET utf8;" | mysql -u root -p

# create a MySQL account, other than root, to be used by django
echo "CREATE USER '$MYSQL_USER'@'localhost' IDENTIFIED BY '$MYSQL_USER_PASSWORD';FLUSH PRIVILEGES;" | sudo mysql
echo "CREATE USER '$MYSQL_USER'@'localhost' IDENTIFIED BY '$MYSQL_USER_PASSWORD';FLUSH PRIVILEGES;" | mysql -u root -p
echo "GRANT ALL PRIVILEGES ON *.* TO '$MYSQL_USER'@'localhost' WITH GRANT OPTION;" | sudo mysql
echo "GRANT ALL PRIVILEGES ON *.* TO '$MYSQL_USER'@'localhost' WITH GRANT OPTION;" | mysql -u root -p

# if on docker localhost will not work so use the docker IP or '%':


# confirm new account can login to new DB
echo " -u $MYSQL_USER -p $MYSQL_USER_PASSWORD $MYSQL_DATABASE" | mysql

# if you delete any user don't forget to FLUSH PRIVILEGES;

In [ ]:
# prepare the app and the virtualenv 

# login as non-root account ambition

# create and source the virtualenv `ambition`
python3 -m venv ~/.venvs/$VENV
source ~/.venvs/$VENV/bin/activate

# pull the app
cd ~/ && git clone $REPO app

# install requirements into the virtualenv
cd ~/app \
&& pip install -r requirements/stable.txt
&& pip install -e .

# Important: the `.env` file is not part of the REPO
# Open another terminal and copy the apps `.env` file to the app root
# assumed coming from your machine, for example:
echo "scp ~/source/ambition/.envs/.local $APP_USER@$APP_HOST:~/app/.env"
scp ~/source/ambition/.envs/.local <app_user>@<app_host>:~/app/.env

# IMPORTANT: inspect the .env variables and edit as required
# NOTE: DATABASE_URL password must be escaped
# set the database name in MYSQL_DATABASE and DATABASE_URL
# See https://github.com/joke2k/django-environ/blob/develop/README.rst#tips
# >>> urllib.parse.quote('my_password')
vi ~/app/.env

# create the export and static folders
mkdir -p $DJANGO_ETC_FOLDER
mkdir -p $DJANGO_EXPORT_FOLDER
mkdir -p $DJANGO_KEY_FOLDER
mkdir -p $DJANGO_LOG_FOLDER
mkdir -p $DJANGO_STATIC_FOLDER

# copy encryption keys into $DJANGO_KEY_FOLDER
# or if just testing set DJANGO_AUTO_CREATE_KEYS=True
# run python manage.py check
# then set DJANGO_AUTO_CREATE_KEYS=False

# check
# since you have not migrated the DB you should see something like this:
# "django.db.utils.ProgrammingError: (1146, "Table 'ambition.edc_lab_boxtype' doesn't exist")"
cd ~/app \
&& python manage.py check

# now migrate (takes a while ...)
python manage.py migrate

# collect static
# if AWS_ENABLED, will test the connection
python manage.py collectstatic

# import holiday file, check the .env to make sure this is correct
python manage.py import_holidays

# import randomization list file, check the .env to make sure this is correct!!
# You need to manually copy a randomization list to $DJANGO_ETC_FOLDER
# given the name $DJANGO_RANDOMIZATION_LIST in `.env`.
python manage.py import_randomization_list

# run check again
# you should have no errors
# "System check identified no issues (0 silenced)."
python manage.py check

# create a super user
python manage.py createsuperuser

# Now try runserver. be sure PORT 8000 is open
# if you get "Invalid HTTP_HOST header: ..." check the `.env` file DJANGO_ALLOWED_HOSTS
# and add your DOMAIN or IP
# IMPORTANT: If you plan to continue with the next section, don't enter any data.
python manage.py runserver 0.0.0.0:8000


## Basic production deploy using NGINX/GUNICORN (DEBUG=False)
Deploy onto an Ubuntu 18.04 server

(continued from above)


In [ ]:
# Since DEBUG=True above, some variables from the `.env` file were ignored
# Now set DEBUG=False in the .env file

# Since DEBUG=False, the app now looks for the 
# encryption keys in DJANGO_KEY_FOLDER. 
# create DJANGO_KEY_FOLDER
mkdir -p $DJANGO_KEY_FOLDER

# echo command to use to copy keys to this host
echo "scp user* $APP_USER@$APP_HOST:$DJANGO_KEY_FOLDER"

# open another terminal and use the above SCP command
# to copy encryption keys to this folder from your encryption key folder
# for example
> scp user* ambition@206.189.16.89:~/.etc/ambition/crypto_fields

# if you run runserver now, the pages will be rendered without static files
# as expected.
python manage.py runserver 0.0.0.0:8000

# notice that the app created django_crypto_fields file
ls -la $DJANGO_ETC_FOLDER

In [ ]:
# setup gunicorn
source ~/.venvs/$VENV/bin/activate
pip install gunicorn

# start gunicorn and try browsing
# again, pages will be rendered without static files
gunicorn --bind 0.0.0.0:8000 ambition.wsgi

# kill gunicorn and deactivate the virtualenv

# config systemd
# create the gunicorn socket
sudo nano /etc/systemd/system/gunicorn.socket
"""
[Unit]
Description=gunicorn socket

[Socket]
ListenStream=/run/gunicorn.sock

[Install]
WantedBy=sockets.target
"""

# create the gunicorn server, replace as needed
sudo nano /etc/systemd/system/gunicorn.service
"""
[Unit]
Description=gunicorn daemon
Requires=gunicorn.socket
After=network.target

[Service]
User=ambition
Group=www-data
WorkingDirectory=/home/ambition/app
ExecStart=/home/ambition/.venvs/ambition/bin/gunicorn \
          --access-logfile - \
          --workers 3 \
          --bind unix:/run/gunicorn.sock \
          ambition.wsgi:application"

[Install]
WantedBy=multi-user.target          

"""

# start and enable
sudo systemctl start gunicorn.socket
sudo systemctl enable gunicorn.socket

# check status
sudo systemctl status gunicorn.socket

# check the socket file
file /run/gunicorn.sock
# Output
/run/gunicorn.sock: socket

# check logs
sudo journalctl -u gunicorn.socket

# if problems, fix and reload
sudo systemctl daemon-reload
sudo systemctl restart gunicorn

In [ ]:
# nginx
sudo nginx -t
sudo systemctl start nginx

In [ ]:
# unset the ENV variables
unset APP_NAME
unset APP_USER
unset DJANGO_ETC_FOLDER
unset DJANGO_EXPORT_FOLDER
unset DJANGO_KEY_FOLDER
unset DJANGO_STATIC_FOLDER
unset MYSQL_DATABASE
unset MYSQL_USER
unset MYSQL_USER_PASSWORD
unset REPO
unset VENV


In [ ]:
sudo docker run -it --rm --name certbot \
-v "/etc/letsencrypt:/etc/letsencrypt" \
-v "/var/lib/letsencrypt:/var/lib/letsencrypt" \
certbot/certbot certonly \
--dns-digitalocean \
--dns-digitalocean-credentials ~/.secrets/certbot/digitalocean.ini \
--dns-digitalocean-propagation-seconds 60 \
-d clinicedc.org